# Introduction: 

words

In [132]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Data Collection:

We collected our data from the IMDB dataset. There were many options of different datasets but we decided to go with the "title.ratings" and "title.basics" datasets because of the average rating and genre columns (respoectfully) present in each set. This enables us to cross examine and check the possible correlation between high rating and particular genres.  

In [172]:
basics_data= pd.read_csv('title.basics.data.tsv', sep='\t')
rating_data = pd.read_csv('title.ratings.data.tsv', sep='\t')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [179]:
data = basics_data.merge(rating_data,left_on='tconst', right_on='tconst')
data = data[data["titleType"] == "movie"]
data

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,5.9,155
259,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama",6.0,39
337,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,4.7,7
371,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama",6.1,593
391,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama,4.5,14
...,...,...,...,...,...,...,...,...,...,...,...
1087195,tt9916132,movie,The Mystery of a Buryat Lama,The Mystery of a Buryat Lama,0,2018,\N,94,"Biography,Documentary,History",3.6,7
1087196,tt9916160,movie,Drømmeland,Drømmeland,0,2019,\N,72,Documentary,6.5,40
1087204,tt9916362,movie,Coven,Akelarre,0,2020,\N,90,"Drama,History,Horror",5.9,125
1087207,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,\N,\N,"Adventure,History,War",4.1,11


# Data Processesing:

## Tidying Data:

Since we asking our question concerning movies only, we isolate it from the dataset we collected. We then replace "\N" IMDB's null type to NaN. We don't really care about the length, end year and possible adult nature of the move so remove those columns as well. Having NaN, in the columns we want to do analysis would be cumbersome so we also remove rows within the data that contain NaN specifically in the genres, averageRating columns. We also noticed some indescrepencies with the types of the data rows. 

To find the average rating for each genre, we will use this equation:

$\frac {\sum {averageRating * numVotes}} {\sum {numVotes}}$

In [180]:
data = data.replace("\\N", np.nan)
#data= data.drop(columns="isAdult")
data.columns
data= data.reset_index()

In [181]:
data = data.dropna(subset=["genres","averageRating"])
data

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance,5.9,155
1,259,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,NaN,NaN,"Biography,Drama",6.0,39
3,371,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Biography,Crime,Drama",6.1,593
4,391,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,NaN,Drama,4.5,14
5,399,tt0000630,movie,Hamlet,Amleto,0,1908,NaN,NaN,Drama,3.2,11
...,...,...,...,...,...,...,...,...,...,...,...,...
254976,1087195,tt9916132,movie,The Mystery of a Buryat Lama,The Mystery of a Buryat Lama,0,2018,NaN,94,"Biography,Documentary,History",3.6,7
254977,1087196,tt9916160,movie,Drømmeland,Drømmeland,0,2019,NaN,72,Documentary,6.5,40
254978,1087204,tt9916362,movie,Coven,Akelarre,0,2020,NaN,90,"Drama,History,Horror",5.9,125
254979,1087207,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,NaN,NaN,"Adventure,History,War",4.1,11


### Keep Genres The Same

Movies can have multiple genres which poses a problem. We will first analyze the data by ignoring this fact, essentially treat all the different "strings" in the genres column of our data set as different genrs.

In [182]:
data['expectedValue'] = data['averageRating'] *data["numVotes"]
grouped_genres = data[['genres','numVotes', 'expectedValue']]
grouped_genres  = grouped_genres .groupby(['genres']).sum()
grouped_genres ['averageRating'] = grouped_genres ['expectedValue']/grouped_genres['numVotes']
grouped_genres = grouped_genres.sort_values(by=['averageRating'], ascending=False)
grouped_genres 

,numVotes,expectedValue,averageRating
genres,,,
"Comedy,Documentary,Fantasy",5,47.0,9.400000
"Family,News",7,65.8,9.400000
"Documentary,Sport,Western",5,44.0,8.800000
"Comedy,Drama,Reality-TV",15,132.0,8.800000
"Documentary,Drama,Reality-TV",19,165.3,8.700000
...,...,...,...
"History,Sci-Fi,Thriller",312,842.4,2.700000
"Comedy,History,Horror",292,741.2,2.538356
"Horror,Reality-TV,Thriller",262,550.2,2.100000


### Seperate Individual Genres 

Now we will analyze the data by seperating each distinct genre from a movie. If a movie has 3 genres such as Comedy,Documentary,and Fantasy, we will "triple count" the movie by putting its statistics in those three different genres.

In [208]:
import re
from copy import deepcopy

temp = []
i = 0
for row in data.iterrows():
    genres = re.split(",",row[1][9])
    for g in genres:
        row2 = deepcopy(row)
        row2[1][9] = g
        temp.append(row2[1])
        
data2 = pd.DataFrame(data = temp)    
data2

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,expectedValue
0,8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance,5.9,155,914.5
1,259,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,NaN,NaN,Biography,6.0,39,234.0
1,259,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,NaN,NaN,Drama,6.0,39,234.0
3,371,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,Biography,6.1,593,3617.3
3,371,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,Crime,6.1,593,3617.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
254978,1087204,tt9916362,movie,Coven,Akelarre,0,2020,NaN,90,Horror,5.9,125,737.5
254979,1087207,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,NaN,NaN,Adventure,4.1,11,45.1
254979,1087207,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,NaN,NaN,History,4.1,11,45.1
254979,1087207,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,NaN,NaN,War,4.1,11,45.1


In [209]:
individual_genres = data2[['genres','numVotes', 'expectedValue']]
individual_genres = individual_genres.groupby(['genres']).sum()
individual_genres['averageRating'] = individual_genres['expectedValue']/individual_genres['numVotes']
individual_genres = individual_genres.sort_values(by=['averageRating'], ascending=False)
individual_genres

,numVotes,expectedValue,averageRating
genres,,,
War,17818934,1.364557e+08,7.657906
News,365620,2.797777e+06,7.652145
Film-Noir,2990130,2.282448e+07,7.633274
Biography,55219144,4.167315e+08,7.546866
Western,8397266,6.320368e+07,7.526697
History,25522252,1.903967e+08,7.460029
Documentary,10712638,7.882246e+07,7.357894
Animation,48094691,3.535094e+08,7.350279
Drama,419619224,3.074662e+09,7.327268
